# Introduction

This script is used to gather the necessary data.
Things to be fixed:
* Token still needs to be refreshed by hand
* Choose time range yourself by inserting the right parameters
* Choose things you want to investigate
* Choose metrics of interest

# Initialization

## Import libraries

In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import json
from datetime import datetime
from datetime import timedelta  
import time
import sys
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

pd.options.mode.chained_assignment = None 

## Set up parameters

In [2]:
##### Choose scope #####
scopeID = 'cot.smart_lighting'

In [3]:
##### Insert token and generate URL #####
myToken = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIwUnhaVzd0N1d5aEczaEo3cUhoM3hQa3MzbkthTUE5Zy04SnozY2trQ3EwIn0.eyJqdGkiOiJhODgyMDAwZS04MWYwLTQ2MjAtODgxNy05YjI2ZjZiNjY1YWQiLCJleHAiOjE1NTQyMjY0NjQsIm5iZiI6MCwiaWF0IjoxNTU0MjI1ODY0LCJpc3MiOiJodHRwczovL2lkbGFiLWlvdC50ZW5ndS5pby9hdXRoL3JlYWxtcy9pZGxhYi1pb3QiLCJhdWQiOiJwb2xpY3ktZW5mb3JjZXIiLCJzdWIiOiI3NGVjNTQzYi03Yjc1LTQ1ZGItOWExNy0xMDY5OTlmYmU3OWEiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzd2FnZ2VyLXVpIiwibm9uY2UiOiIyMjNkNjQ3My0wZmU3LTQxNmItODNiNS0wN2I3NmRjYTlkOTgiLCJhdXRoX3RpbWUiOjE1NTQyMjU4NjMsInNlc3Npb25fc3RhdGUiOiIzMjBkZDkzZS1kZDEyLTRmNzktYjA0ZC0zZTlhNDlkMmNiNGMiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vaWRsYWItaW90LnRlbmd1LmlvIiwiaHR0cDovL2xvY2FsaG9zdDo1NTU1Il0sInJlc291cmNlX2FjY2VzcyI6eyJwb2xpY3ktZW5mb3JjZXIiOnsicm9sZXMiOlsidXNlcjp2aWV3Il19fSwiYXV0aG9yaXphdGlvbiI6eyJwZXJtaXNzaW9ucyI6W3sicnNpZCI6IjQxOGFiMGYzLWE5NTQtNGEwMS04ZTdlLWFlZGQzN2VjZTcyMyIsInJzbmFtZSI6ImRhdGE6c2NvcGVkOnZpZXcifV19LCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIGNvdC1zY29wZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwicmlnaHRzIjpbXSwibmFtZSI6IkplZmYgR2V1ZGVucyIsImdyb3VwcyI6WyIvYWxsLXVzZXJzIiwiL2NvdC90aGVzaXMiXSwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2V1ZGVucy5qZWZmQGdtYWlsLmNvbSIsImdpdmVuX25hbWUiOiJKZWZmIiwiZmFtaWx5X25hbWUiOiJHZXVkZW5zIiwiZW1haWwiOiJnZXVkZW5zLmplZmZAZ21haWwuY29tIiwicGljdHVyZSI6Imh0dHBzOi8vbGg1Lmdvb2dsZXVzZXJjb250ZW50LmNvbS8tRTYxVzBvTmFlOUkvQUFBQUFBQUFBQUkvQUFBQUFBQUFCd2MvUExDNy1mNTdnek0vcGhvdG8uanBnP3N6PTUwIn0.nhpHfml-_HOvvc2r3y5VCl0xPBz33_kcB4cn1iDSdGF-dkwi6lkSPSTsNxltxU9d8ZMtIviMcU0ha4dt87NJ7RZZRCFUcmlR5GjKuD1qA5NbmvoTSDums8_kAWj-LrBFQqsFxa8TwoQK5sLLMYTDbBt8CYtiNGaZDUifEg9t_yc8Mck3N2QBG-G28_H4zhDPmQvEP0Kg_X8V7nn5a8f2Qeh20XSKcMUxLn80oHXfaC5O0P-CS2OA_TeKaN0UQdz6EjkKtQlsIiKRAZ8Syevcl77x3ZfWyTnlcQ0-BKKD8nj5TVf55eSzVID6HZnt-9_LVBBOB4CEnm35U-zMh8exFA'

In [4]:
##### Specify time range #####
##### Adjust this time range to specify which period you want to query - Mostly I use periods of one month #####
startDate = datetime(2019, 3, 1, 1, 0)
endDate = datetime(2019, 4, 1, 1, 0)
startTime = int(startDate.timestamp())*1000;  #print(startTime)
endTime = int(endDate.timestamp())*1000;  #print(endTime)

# Weather station

In [5]:
thingID = 'davis.davis.weather.1'

### Choose metrics

In [6]:
metricsList = ['environment.light','environment.temperature','weather.uv','environment.relativehumidity','weather.pressure', 'weather.rainrate', 'weather.windspeed']
# metricsList = ['weather.pressure', 'weather.rainrate', 'weather.windspeed']

### Run query

In [7]:
for i in range(0,len(metricsList)):
    print(metricsList[i]+'_'+startDate.strftime('%Y-%m'))
    metricID = metricsList[i]

    myUrlQuery = 'https://idlab-iot.tengu.io/api/v1/scopes/{scope}/query/{metric}/events?from={start}&to={end}&things={thing}&orderByTime=asc'\
        .format(scope = scopeID, metric = metricID, start=startTime, end=endTime, thing=thingID)

    headers = {
        'accept': 'application/json',
        'authorization': 'Bearer {0}'.format(myToken),
    }

    response = requests.get(myUrlQuery, headers=headers)
    #print(response.content)
    data = response.json() 
    column_names = data['columns']; #print(column_names)
    values = data['values']
    df = pd.DataFrame.from_dict(values)
    df.columns = column_names
    weather_data = df[['time','value']]
#     print(weather_data)
#     path=r'C:\Users\JeffG\Desktop\Case 1 - data\weather_data'
    path=r'D:\Documents\Thesis\Case 1 - data\weather_data'
    weather_data.to_csv(path+'\\'+metricsList[i]+'_'+startDate.strftime('%Y-%m')+'.csv')

environment.light_2019-03
environment.temperature_2019-03
weather.uv_2019-03
environment.relativehumidity_2019-03
weather.pressure_2019-03
weather.rainrate_2019-03
weather.windspeed_2019-03


# Battery pack

In [11]:
#thingID = 'munisense.msup1g30034' # west facing
thingID = 'munisense.msup1i70124' # south facing
# thingID = 'munisense.msup1h90115' # east facing

### Choose metrics

In [12]:
# metricsList = ['Power.BatteryState']
metricsList = ['Power.BatteryState','Power.BatteryVoltHR','Power.PercentageRemaining','Power.TimeRemaining']

### Run query

In [13]:
for i in range(0,len(metricsList)):
    print(metricsList[i]+'_'+startDate.strftime('%Y-%m'))
    metricID = metricsList[i]

    myUrlQuery = 'https://idlab-iot.tengu.io/api/v1/scopes/{scope}/query/{metric}/events?from={start}&to={end}&things={thing}&orderByTime=asc'\
        .format(scope = scopeID, metric = metricID, start=startTime, end=endTime, thing=thingID)

    headers = {
        'accept': 'application/json',
        'authorization': 'Bearer {0}'.format(myToken),
    }

    response = requests.get(myUrlQuery, headers=headers)
    response.content
    data = response.json(); #print(data)
    column_names = data['columns']; #print(column_names)
    values = data['values']
    df = pd.DataFrame.from_dict(values)
    df.columns = column_names
    battery_data = df[['time','value']]
#     print(battery_data)
    path=r'D:\Documents\Thesis\Case 1 - data\battery_data_2' #vaste pc
#     path=r'C:\Users\JeffG\Desktop\Case 1 - data\battery_data_2' #Laptop
    battery_data.to_csv(path+'\\'+metricsList[i]+'_'+startDate.strftime('%Y-%m')+'.csv')

Power.BatteryState_2019-03
Power.BatteryVoltHR_2019-03
Power.PercentageRemaining_2019-03
Power.TimeRemaining_2019-03


In [ ]:
def load_voltage(startDate, endDate, thingID, myToken):
    ##### Specify time range #####
    ##### Adjust this time range to specify which period you want to query - Mostly I use periods of one month #####
    startTime = int(startDate.timestamp())*1000;  #print(startTime)
    endTime = int(endDate.timestamp())*1000;  #print(endTime)

    ##### Choose scope #####
    scopeID = 'cot.smart_lighting'

    ##### Specify query #####
    metricID = 'Power.BatteryVoltHR'
    myUrlQuery = 'https://idlab-iot.tengu.io/api/v1/scopes/{scope}/query/{metric}/events?from={start}&to={end}&things={thing}&orderByTime=asc'\
        .format(scope = scopeID, metric = metricID, start=startTime, end=endTime, thing=thingID)

    headers = {
        'accept': 'application/json',
        'authorization': 'Bearer {0}'.format(myToken),
    }

    ##### Execute query #####
    response = requests.get(myUrlQuery, headers=headers)

    try:
        data = response.json(); 
        column_names = data['columns']; 
        values = data['values']
        df = pd.DataFrame.from_dict(values)
        df.columns = column_names
        df["value"] = pd.to_numeric(df.value, errors='coerce')

        ##### Clean up dataFrame #####
        # Rename the column
        df.rename(columns={'value':'Power.BatteryVoltHR'}, inplace=True)
        # Set timestamp as index
        df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.time, unit='ms')))
        df.drop(['time'], axis=1,inplace=True)
        message = 'All good'

        return df, message

    except ValueError:
        df = None
        errorMessage = "Decoding JSON has failed. Probably the token has expired"
        return df, errorMessage
    

In [ ]:
endDate = datetime.now()
startDate = endDate - timedelta(3/12*365)
thingID = 'munisense.msup1h90115' # east facing
myToken = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIwUnhaVzd0N1d5aEczaEo3cUhoM3hQa3MzbkthTUE5Zy04SnozY2trQ3EwIn0.eyJqdGkiOiI1Y2Y3NjAyNi1mYTJhLTRmZTYtYWU5ZC02NDU1MjI3ZWVhZDciLCJleHAiOjE1NTQxNDc0MDQsIm5iZiI6MCwiaWF0IjoxNTU0MTQ2ODA0LCJpc3MiOiJodHRwczovL2lkbGFiLWlvdC50ZW5ndS5pby9hdXRoL3JlYWxtcy9pZGxhYi1pb3QiLCJhdWQiOiJwb2xpY3ktZW5mb3JjZXIiLCJzdWIiOiI3NGVjNTQzYi03Yjc1LTQ1ZGItOWExNy0xMDY5OTlmYmU3OWEiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzd2FnZ2VyLXVpIiwibm9uY2UiOiJhMzRmMmE1Yy02NzgzLTRmYmMtOTNkMy1kOWIyNDhhNTFmZmEiLCJhdXRoX3RpbWUiOjE1NTQxNDY4MDMsInNlc3Npb25fc3RhdGUiOiIwOTQyN2VkMC02YTg0LTRlZDktOTYyOC04NzkyMjAzZTM2ODkiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vaWRsYWItaW90LnRlbmd1LmlvIiwiaHR0cDovL2xvY2FsaG9zdDo1NTU1Il0sInJlc291cmNlX2FjY2VzcyI6eyJwb2xpY3ktZW5mb3JjZXIiOnsicm9sZXMiOlsidXNlcjp2aWV3Il19fSwiYXV0aG9yaXphdGlvbiI6eyJwZXJtaXNzaW9ucyI6W3sicnNpZCI6IjQxOGFiMGYzLWE5NTQtNGEwMS04ZTdlLWFlZGQzN2VjZTcyMyIsInJzbmFtZSI6ImRhdGE6c2NvcGVkOnZpZXcifV19LCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIGNvdC1zY29wZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwicmlnaHRzIjpbXSwibmFtZSI6IkplZmYgR2V1ZGVucyIsImdyb3VwcyI6WyIvYWxsLXVzZXJzIiwiL2NvdC90aGVzaXMiXSwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2V1ZGVucy5qZWZmQGdtYWlsLmNvbSIsImdpdmVuX25hbWUiOiJKZWZmIiwiZmFtaWx5X25hbWUiOiJHZXVkZW5zIiwiZW1haWwiOiJnZXVkZW5zLmplZmZAZ21haWwuY29tIiwicGljdHVyZSI6Imh0dHBzOi8vbGg1Lmdvb2dsZXVzZXJjb250ZW50LmNvbS8tRTYxVzBvTmFlOUkvQUFBQUFBQUFBQUkvQUFBQUFBQUFCd2MvUExDNy1mNTdnek0vcGhvdG8uanBnP3N6PTUwIn0.hz9g4pUg-_xFRWum7BR4hhjCW3mQEaBgQ3VfJe-IRoDoIa6FmDQ7J-AQ7W4PbkSr1cXw4sy94B4fL_6GYoV2QwdqJNEqipCV0YVvfNt7UmLeyLMLDPQbZIrRMu-O1k-JQ67nroFlPCQKrIdB5pvfqYHQzpEkp6niAQu_Mypm2Hgz7ulSC0xlgY3rqmkIo1VhpRNULEmS1B19_91DdFgmADX6843HtOlJCll4xJ-hLfpnFBNejczNX8gQ9ImfRzJxSYEAAwETQLVgDXu_gQyeo7z4P0OuWx51SDAhhHFp9f5tfjLDvb86Kk2sZxSbdsH9sMANAfzduMGjFVfZIzxOFA'

df = load_voltage(startDate, endDate, thingID, myToken)

In [ ]:
df['moving_average'] = df['Power.BatteryVoltHR'].rolling(window_size).mean()
df['residual'] = df['Power.BatteryVoltHR'] - df['moving_average']

# Label anomalies

In [ ]:
def label_anomaly_moving_std (row):
    if (row['Power.BatteryVoltHR'] > (row['moving_average'] + (sigma * row['moving_std']))) | (row['Power.BatteryVoltHR'] < (row['moving_average'] - (sigma * row['moving_std']))):
        return 1
    else:
        return 0

def label_anomaly_fixed_std (row, fixed_std):
    if (row['Power.BatteryVoltHR'] > (row['moving_average'] + (sigma * fixed_std))) | (row['Power.BatteryVoltHR'] < (row['moving_average'] - (sigma * fixed_std))):
        return 1
    else:
        return 0

# Low pass filtering

In [ ]:
def low_pass_filtering(df, window_size, sigma, fixed_std):
    # Calculate moving average
    df['moving_average'] = df['Power.BatteryVoltHR'].rolling(window_size).mean()

    # Calculate anomalies
    df['residual'] = df['Power.BatteryVoltHR'] - df['moving_average']
    df['moving_std'] = df['Power.BatteryVoltHR'].rolling(window_size).std()
    df.dropna(inplace=True)
    df['anomaly_flag_moving_std']=df.apply (lambda row: label_anomaly_moving_std(row), axis=1)
    df['anomaly_flag_fixed_std']=df.apply (lambda row: label_anomaly_fixed_std(row, fixed_std), axis=1)

    moving_anomalies = df[df['anomaly_flag_moving_std']==1]
    fixed_anomalies = df[df['anomaly_flag_fixed_std']==1]
    
    return df

In [ ]:
df = low_pass_filtering(df, 256, 3, fixed_std)
moving_anomalies = df[df['anomaly_flag_moving_std']==1]
fixed_anomalies = df[df['anomaly_flag_fixed_std']==1]

# Saving data to SQL database

## Load libraries

In [ ]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

In [ ]:
def initialize_database():
    user = 'root'
    passw = 'root'
    host =  'localhost'  # either localhost or ip e.g. '172.17.0.2' or hostname address 
    port = 3306 
    database = 'smart_lighting'
    engine = create_engine('mysql+mysqlconnector://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database , echo=False)
    return engine

def load_voltage_from_database(thingID, startDate, endDate, engine):
    table_name = 'battery_voltage'
    start_date = startDate.strftime("%Y-%m-%d %H:%M:%S")
    end_date = endDate.strftime("%Y-%m-%d %H:%M:%S")
    query = ''' SELECT * 
                FROM {} 
                WHERE (sourceId = '{}') AND (time between '{}' and '{}')'''.format(table_name, thingID, start_date, end_date)
    df = pd.read_sql(query, engine, index_col='time')
    return df

In [ ]:
endDate = datetime.now()
startDate = endDate - timedelta(9/12*365)
myToken = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIwUnhaVzd0N1d5aEczaEo3cUhoM3hQa3MzbkthTUE5Zy04SnozY2trQ3EwIn0.eyJqdGkiOiIzOGZkM2UwMi1mOTE4LTQzNTMtYmQwZi1hNTE0MWUzZTc5ZjUiLCJleHAiOjE1NTQxMzIwMjEsIm5iZiI6MCwiaWF0IjoxNTU0MTMxNDIxLCJpc3MiOiJodHRwczovL2lkbGFiLWlvdC50ZW5ndS5pby9hdXRoL3JlYWxtcy9pZGxhYi1pb3QiLCJhdWQiOiJwb2xpY3ktZW5mb3JjZXIiLCJzdWIiOiI3NGVjNTQzYi03Yjc1LTQ1ZGItOWExNy0xMDY5OTlmYmU3OWEiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzd2FnZ2VyLXVpIiwibm9uY2UiOiIyNTUwNDA4YS03NWRjLTRiM2EtYjM2Yi1iYzQ5ZjU3MzhlMWYiLCJhdXRoX3RpbWUiOjE1NTQxMzA5MDAsInNlc3Npb25fc3RhdGUiOiIzNjFjMmUzYi1kMTRlLTRhZTMtODU1Ni1kOTA4OTFkMGM4OWUiLCJhY3IiOiIwIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vaWRsYWItaW90LnRlbmd1LmlvIiwiaHR0cDovL2xvY2FsaG9zdDo1NTU1Il0sInJlc291cmNlX2FjY2VzcyI6eyJwb2xpY3ktZW5mb3JjZXIiOnsicm9sZXMiOlsidXNlcjp2aWV3Il19fSwiYXV0aG9yaXphdGlvbiI6eyJwZXJtaXNzaW9ucyI6W3sicnNpZCI6IjQxOGFiMGYzLWE5NTQtNGEwMS04ZTdlLWFlZGQzN2VjZTcyMyIsInJzbmFtZSI6ImRhdGE6c2NvcGVkOnZpZXcifV19LCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIGNvdC1zY29wZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwicmlnaHRzIjpbXSwibmFtZSI6IkplZmYgR2V1ZGVucyIsImdyb3VwcyI6WyIvYWxsLXVzZXJzIiwiL2NvdC90aGVzaXMiXSwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2V1ZGVucy5qZWZmQGdtYWlsLmNvbSIsImdpdmVuX25hbWUiOiJKZWZmIiwiZmFtaWx5X25hbWUiOiJHZXVkZW5zIiwiZW1haWwiOiJnZXVkZW5zLmplZmZAZ21haWwuY29tIiwicGljdHVyZSI6Imh0dHBzOi8vbGg1Lmdvb2dsZXVzZXJjb250ZW50LmNvbS8tRTYxVzBvTmFlOUkvQUFBQUFBQUFBQUkvQUFBQUFBQUFCd2MvUExDNy1mNTdnek0vcGhvdG8uanBnP3N6PTUwIn0.d-QVLWkYSYwbBBsFaub3onbnrF5ikyrVhu6z4jCIXQm7g1-Khc-SFWcD802IM0QppseSQziFD0YnhrVTD4SipUncTiJBjT2db235pKk8715tLH1PvijCDnPiTESs8IB6EZWEdBuiInEbgGN-3shhixfWySTA6L1W0XpkHvGpYdbDgIaTSz_uHn6Ebe2DpLm6Nu5MR1teR1ojuz_jLZVC0KNr75iUSfINCecOo2blbzQqVP_E1B9vk2b9NBiAS5vwX_j3r8kw-NX-xbK9qVRMQl1ImC38QBEveqNDE2qIEK3kCP7VKBsZX2t1M5KvpZ3q7f6oWRm9lQc5QMoAh5ThGw'
things = ['munisense.msup1g30034', 'munisense.msup1i70124', 'munisense.msup1h90115', 'munisense.msup1h90103']
for thingID in things:
    df = load_voltage(startDate, endDate, thingID, myToken)
    df.to_sql(name='battery_voltage', con=engine, if_exists = 'append', index=True)

In [ ]:
endDate = datetime.now(); print(endDate)
startDate = endDate - timedelta(8/12*365); print(startDate)

df1 = load_voltage_from_database(thingID, startDate, endDate, engine)
startDate = find_last_datapoint(thingID)
df2 = load_voltage_from_database('munisense.msup1h90103', startDate, endDate, engine)
